In [ ]:
'''
检验已采集至汽修宝APP的车型图片url后缀是否唯一独立
'''
import pymysql
import re

# 链接汽修宝采集图片数据库
def select_pics_url_from_MySQL(table_name):
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='MySQL', charset='utf8')
    cur = conn.cursor()
    cur.execute('USE autohome')
    sql = '''
        SELECT DISTINCT `model_id`,`img_url`,`oss_url`
        FROM %s
        WHERE `type` = 1 AND `oss_url` != '';
    ''' % table_name
    cur.execute(sql)
    result = cur.fetchall()
    cur.close()
    conn.close()
    result_list = []
    if result:
        for i in result:
            result_list.append(list(i))
    return result_list

# 正则处理图片url,提取后缀关键code
def re_get_url_code(url):
    re_result = re.compile(r'/[\w_-]+(.png|.jpg)$').search(url)
    if re_result:
        re_result = re_result.group()
        # 处理前部/
        re_result = re_result.replace('/y_','').replace('/w_','').replace('/t_','').replace('/','')
        # 处理可能出现的autohomecar__
        re_result = re_result.replace('autohomecar__','')
        # 处理后部.jpg .png
        re_result = re_result.replace('.jpg','').replace('.png','')
        return re_result
    else:
        print('正则表达式无法匹配')
        return None

table_name = 'vehicle_pics'
select_pics_url = select_pics_url_from_MySQL(table_name)
            


In [ ]:
# 新增url_code至汽修宝列表
qxb_list = []
for i in select_pics_url:
    re_result = re_get_url_code(i[1])
    if re_result:
        i.append(re_result)
        qxb_list.append(i)
    else:
        print('正则处理有误')

In [ ]:
# 汽修宝数据转字典
qxb_dict = {}
for j in qxb_list:
    if j[2] in qxb_dict:
        print('oss_url有重复',j)
    else:
        # 为每个oss_url设置一个字典，并给定model_id,img_url,url_code
        qxb_dict[j[2]]={}
        qxb_dict[j[2]]['model_id'] = j[0]
        qxb_dict[j[2]]['img_url'] = j[1]
        qxb_dict[j[2]]['url_code'] = j[-1]
print(len(qxb_dict))
        


In [ ]:
# 数据库查询汽车之家多外观颜色图片url
def select_more_color_url_from_MySQL():
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='MySQL', charset='utf8')
    cur = conn.cursor()
    cur.execute('USE autohome_carcolor')
    # 查多外观颜色图片url
    sql = '''
        SELECT 
            cars_pics_cg1_info.id,
            cars_pics_cg1_info.vehicle_id,
            cars_pics_cg1_info.color_id,
            cars_pics_cg1_info.smallpic,
            carcolor_info.color_name
        FROM cars_pics_cg1_info
        LEFT JOIN carcolor_info
        ON 
            cars_pics_cg1_info.vehicle_id = carcolor_info.vehicle_id
            AND cars_pics_cg1_info.color_id = carcolor_info.color_id;
    '''
    cur.execute(sql)
    result = cur.fetchall()
    cur.close()
    conn.close()
    result_list = []
    if result:
        for i in result:
            result_list.append(list(i))
    return result_list

result_list = select_more_color_url_from_MySQL()
print(len(result_list))


In [ ]:
# 新增url_code至汽车之家列表
autohome_list = []
for i in result_list:
    re_result = re_get_url_code(i[3])
    if re_result:
        i.append(re_result)
        autohome_list.append(i)
    else:
        print('正则处理有误')

print(len(autohome_list))

In [ ]:
# 汽车之家数据转字典
autohome_dict = {}
for j in autohome_list:
    if j[0] in autohome_dict:
        print('id有重复',j)
    else:
        # 为每个序号id设置一个字典，并给定vehicle_id,color_id,smallpic,color_name,url_code
        autohome_dict[j[0]]={}
        autohome_dict[j[0]]['vehicle_id'] = j[1]
        autohome_dict[j[0]]['color_id'] = j[2]
        autohome_dict[j[0]]['smallpic'] = j[3]
        autohome_dict[j[0]]['color_name'] = j[4]
        autohome_dict[j[0]]['url_code'] = j[-1]
print(len(autohome_dict))
print(autohome_dict[100])

In [ ]:
# 实现匹配
# 匹配结果列表初始化
match_result_lists = []
# 遍历汽车之家字典
for k,v in autohome_dict.items():
    # 遍历汽修宝字典
    for kk,vv in qxb_dict.items():
        # 判断车型ID与url_code是否一致
        if v['vehicle_id'] == vv['model_id'] and v['url_code'] == vv['url_code']:
            num_id = k
            vehicle_id = v['vehicle_id']
            color_id = v['color_id']
            smallpic = v['smallpic']
            color_name = v['color_name']
            img_url = vv['img_url']
            oos_url = kk
            match_result_lists.append([num_id,vehicle_id,color_id,smallpic,color_name,img_url,oos_url])
            print(match_result_lists)

            

In [ ]:
print(len(match_result_lists))